In [1]:
import os
import numpy as np
import pandas as pd
# import tensorflow as tf
# from tensorflow import keras 

# from sklearn.model_selection import train_test_split
# from sklearn.utils import resample

# from tensorflow.keras.models import load_model, Sequential
# from tensorflow.keras.layers import Dense, Dropout, Flatten
# from tensorflow.keras import regularizers
# from tensorflow.keras.callbacks import EarlyStopping

# from dotenv import load_dotenv
# load_dotenv()

In [3]:
final_df = pd.read_csv(r'D:\OneDrive - NITT\Custom_Download\training_data.csv')

In [5]:
final_df['clicker'].value_counts()

0    210492
1      5173
Name: clicker, dtype: int64

In [ ]:
def new_balance(df_train,col):
    df_no = df_train[df_train[col]==0]
    df_yes = df_train[df_train[col]==1]
    del df_train
    if df_yes.shape[0] > df_no.shape[0]:
        yes_downsampled = resample(df_yes, 
                                   replace = False, # sample without replacement
                                   n_samples = len(df_no), # match minority n
                                   random_state = 42)
        resampled = pd.concat([yes_downsampled, df_no])
    elif df_no.shape[0] > df_yes.shape[0]:
        no_downsampled = resample(df_no, replace = False, n_samples = len(df_yes), random_state = 42)
        resampled = pd.concat([no_downsampled, df_yes])
    else:
        resampled = pd.concat([df_yes, df_no])
    return resampled

In [ ]:
# this is a bunch of sql queries that get combined to make the dataframe that will get fed into the model
final_df = pd.read_csv('D:\OneDrive - NITT\Custom_Download\training_data.csv')

# goal is to predict a clicker
final_df = new_balance(final_df,'clicker')

df_train, df_test = train_test_split(final_df, test_size=0.2, random_state = 42)
df_train, df_val = train_test_split(df_train, test_size=0.1, random_state = 42)

train_x = np.asarray(df_train.iloc[:,3:])
train_y = np.asarray(df_train['clicker'])

test_x = np.asarray(df_test.iloc[:,3:])
test_y = np.asarray(df_test['clicker'])

val_x = np.asarray(df_val.iloc[:,3:])
val_y = np.asarray(df_val['clicker'])

model = Sequential([ 
    Flatten(input_dim=train_x.shape[1]),
    Dense(49, activation="relu", name="layer2",kernel_regularizer=regularizers.l1(1e-5)),
    Dense(49, name="layer3"),
    Dense(10, name="layer4"),
    Dense(1,activation='sigmoid', name="layer5")])

model.compile(optimizer='adam', 
    loss='binary_crossentropy', 
    metrics=['mse'])

es = EarlyStopping(monitor='val_loss', mode='min',patience=10, verbose=0)

history=model.fit(train_x,train_y,validation_data=(test_x, test_y), epochs=400,batch_size=100,verbose=0,callbacks=[es])

model.save_weights(f"training_data.h5")

# sql query that results in scoring data        
scoring_df = pd.read_csv('scoring_data.csv')

X = scoring_df.iloc[:,1:].fillna(0)
X = np.asarray(X)

model2 = Sequential([
                    Flatten(input_dim=X.shape[1]),
                    Dense(49, activation="relu", name="layer2",kernel_regularizer=regularizers.l1(1e-5)),
                    Dense(49, name="layer3"),
                    Dense(10, name="layer4"),
                    Dense(1,activation='sigmoid', name="layer5")])


model2.load_weights(f"{list_code}_{from_email}_{network_code}{offer_id}.h5",by_name=True) 

x_raw = model2.predict(X,batch_size=100)
x_predict = model2.predict(X,batch_size=100)

# result is taking the training data vs scoring data and predicting the users probability that they will click
df = pd.concat([scoring_df['email'], pd.DataFrame((x_predict)), pd.DataFrame((x_raw[:, 0]).T)], axis=1)
df.columns = ['email','pred_class', 'pred_prob']
df = df.sort_values(by='pred_prob', ascending=False)